In [47]:
import pandas as pd
import skcriteria as skc
import numpy as np
import matplotlib.pyplot as plt
import json
import yaml
import pprint
from tabulate import tabulate

from skcriteria.preprocessing import invert_objectives, scalers
from skcriteria.madm import simple

def get_id(key):
    criterion_key = {
        'Name' : 'name',
        'Elastic Modulus' : 'elastic_mod',
        'Yield Strength' : 'yield_strength',
        'Cost' : 'cost',
        'Ultimate Strength': 'ult_strength',
        'Machineability' : 'machineability'
    }

    if type(key) == list:
        result = [criterion_key.get(k) for k in key]
        result = [x for x in result if x != None]
        return result

    return criterion_key.get(key)

with open('C:/Users/ttrol/CodingProjects/MatNet/webscraper/results_files/AISI_steels_fakedata.yaml','r') as stream:
    try:
        raw_data = yaml.safe_load(stream)
    except FileNotFoundError:
        raise FileNotFoundError

raw_dataframe = pd.DataFrame(raw_data)


In [66]:
criterions = [ 'Yield Strength', 'Cost', 'Machineability', '', '']
raw_weights = [-10, 3, 4, 4, 3]

formData = {}
for i in range(5):
    if criterions[i] != '' and raw_weights[i] != 0:
        formData[criterions[i]] = raw_weights[i]  

# print(formData)

weights = np.asarray(list(formData.values()))
weights = np.divide(weights,10)


objectives = []
for weight in weights:
    if weight < 0:
        objectives.append(min)
    elif weight > 0:
        objectives.append(max)


criterion_ids = get_id(list(formData.keys()))

df = raw_dataframe.loc[:, criterion_ids]        #type: ignore
# # let's says the first two alternatives are
# # for maximization and the last one for minimization

# we use the built-in function as aliases
row_id =  list(raw_dataframe.loc[:, 'name'])
dm = skc.mkdm(
    df.to_numpy(),
    objectives= objectives,
    weights= weights,
    alternatives = row_id,
    criteria= list(formData.keys())
)
dm


,Yield Strength[▼-1.0],Cost[▲ 0.3],Machineability[▲ 0.4]
"AISI E9310H Steel, normalized, machined to 25 mm (1 in.) rounds, pseudocarburized, 150°C (300°F) temper",1030.0,9.94,49.0
"AISI 4027 Steel, normalized, 14 mm (0.55 in.) round",425.0,0.19,2.0
"AISI 1020 Steel, mock carburized at 915°C (1680°F) for 8 hours, 775°C (1430°F) reheat, water quenched, 175°C (350°F) temper, 100 mm (4 in.) round",290.0,9.11,36.0
"AISI 1095 Steel, oil quenched 800°C (1470°F), 480°C (900°F) temper, 100 mm (4 in.) round",640.0,7.96,14.0
"AISI 4620H Steel, direct quench from pot, carburized, oil quenched, 150°C (300°F) temper",802.0,5.32,71.0
...,...,...,...
"AISI 1060 Steel, oil quenched 845°C (1550°F), 480°C (900°F) temper, 100 mm (4 in.) round",518.0,4.01,57.0
"AISI Grade 18Ni (250) Maraging Steel, 3.18 mm thick sheet, solution annealed at 815°C (1500°F) for 30 minutes, air cooled, aged at 455°C (850°F) for 3 hours",1770.0,4.17,80.0
"AISI 1340H Steel, oil quenched from 830°C (1525°F), 595°C (1100°F) temper, 50 mm (2 in.) round",565.0,4.30,11.0
"AISI 4820 Steel, mock carburized, reheated to 800°C (1470°F), 150°C (300°F) temper, 13 mm (0.5 in.) round",1200.0,6.84,57.0


In [68]:
from skcriteria.preprocessing import invert_objectives, scalers
from skcriteria.madm import simple

#SKC handles maximization or minimization only best , so we invert the third row to be a maximization problem 
#maximization is preferable to minimizationm
inverter = invert_objectives.InvertMinimize()
dmt = inverter.transform(dm)
print(dmt)

#normalizes values
scaler = scalers.SumScaler(target="both")
dmt = scaler.transform(dmt)

dec = simple.WeightedSumModel()
rank = dec.evaluate(dmt)  # we use the tansformed version of the data
rank

                                                    Yield Strength[▲-1.0]  \
AISI E9310H Steel, normalized, machined to 25 m...               0.000971   
AISI 4027 Steel, normalized, 14 mm (0.55 in.) r...               0.002353   
AISI 1020 Steel, mock carburized at 915°C (1680...               0.003448   
AISI 1095 Steel, oil quenched 800°C (1470°F), 4...               0.001563   
AISI 4620H Steel, direct quench from pot, carbu...               0.001247   
...                                                                   ...   
AISI 1060 Steel, oil quenched 845°C (1550°F), 4...               0.001931   
AISI Grade 18Ni (250) Maraging Steel, 3.18 mm t...               0.000565   
AISI 1340H Steel, oil quenched from 830°C (1525...               0.001770   
AISI 4820 Steel, mock carburized, reheated to 8...               0.000833   
AISI 4815H Steel, pseudocarburized and oil quen...               0.000935   

                                                    Cost[▲ 0.3]  \
AISI E93

Alternatives,"AISI E9310H Steel, normalized, machined to 25 mm (1 in.) rounds, pseudocarburized, 150°C (300°F) temper","AISI 4027 Steel, normalized, 14 mm (0.55 in.) round","AISI 1020 Steel, mock carburized at 915°C (1680°F) for 8 hours, 775°C (1430°F) reheat, water quenched, 175°C (350°F) temper, 100 mm (4 in.) round","AISI 1095 Steel, oil quenched 800°C (1470°F), 480°C (900°F) temper, 100 mm (4 in.) round","AISI 4620H Steel, direct quench from pot, carburized, oil quenched, 150°C (300°F) temper","AISI 86B50 Steel, normalized, 13 mm (0.5 in.) round","AISI E9310H Steel, single quenched and tempered at 150°C (300°F), 1.2 mm case depth","AISI 4140H Steel, heat treated, 595°C (1100°F) temper, 25 mm (1 in.) round","AISI 4320 Steel, normalized 895°C (1640°F), air cooled, 50 mm (2 in.) round","AISI 1021 Steel, cold drawn","AISI 4815H Steel, pseudocarburized, reheated to 830°C (1525°F) and oil quenched, 150°C (300°F) temper, 25 mm (1 in.) round","AISI 1010 Steel, hot rolled bar, 19-32 mm (0.75-1.25 in) round or thickness","AISI 1080 Steel, cold drawn, spheroidized annealed, 19-32 mm (0.75-1.25 in) round","AISI Type S15500 (15Cr-5Ni) Precipitation Hardening Stainless Steel transverse direction, condition H1150","AISI 8630H Steel, water quenched 845°C (1550°F), 595°C (1100°F) temper, 100 mm (4 in.) round",AISI Type S14800 Stainless Steel condition SRH1050,"AISI 1035 Steel, water quenched from 845°C (1550°F), 540°C (1000°F) temper","AISI 9255 Steel, oil quenched from 885°C (1625°F), 540°C (1000°F) temper, 25 mm (1 in.) round","AISI 4620H Steel, mock carburized, 150°C (300°F) temper, 100 mm (4 in.) round","AISI 4320H Steel, mock carburized, oil quenched, 230°C (450°F) temper, 13 mm (0.5 in.) round","AISI Type S45500 Stainless Steel, hardened at 540°C (1000°F) for 4 hours, air cooled","AISI 1117 Steel, normalized at 900°C (1650°F)","AISI 1020 Steel, mock carburized at 915°C (1680°F) for 8 hours, 775°C (1430°F) reheat, water quenched, 175°C (350°F) temper, 50 mm (2 in.) round","AISI 1038 Steel, hot rolled round, 19-32 mm (0.75-1.25 in)",AISI Type S21800 Stainless Steel 76 mm annealed bar,"AISI 8620H Steel, normalized, machined to 25 mm, pseudocarburized, reheated to 775°C, 150°C temper","AISI 8630 Steel, water quenched 845°C (1550°F), 480°C (900°F) temper, 13 mm (0.5 in.) round","AISI 1215 Steel, turned and polished, 19-38 mm round","AISI 1132 Steel, hot rolled and cold drawn, 16-38 mm round","AISI Type W2 Tool Steel, water quenched 775°C (1425°F), tempered 500°C (925°F)","AISI 4419 Steel, mock carburized, 150°C (300°F) temper, 100 mm (4 in.) round","AISI 4820 Steel, normalized 860°C (1580°F), 50 mm (2 in.) round","AISI E4340 H Steel, oil quenched 800°C (1470°F), 650°C (1200°F) temper, 100 mm (4 in.) round","AISI 4718H Steel, pseudocarburized, reheated to 765°C (1410°F) and oil quenched, 150°C (300°F) temper, 13.7 mm round","AISI 1012 Steel, cold drawn","AISI 1022 Steel, annealed at 870°C (1600°F), furnace cooled 17°C (31°F) per hour to 675°C, air cooled, 25 mm (1 in.) round","AISI 4150H Steel, oil quenched 830°C (1525°F), 650°C (1200°F) temper, 50 mm (2 in.) round","AISI Type S20910 Stainless Steel, high strength","AISI 1144 Steel, oil quenched 845°C (1550°F), 595°C (1100°F) temper, 25 mm (1 in.) round","AISI E4340 Steel, oil quenched 800°C (1470°F), 540°C (1000°F) temper, 50 mm (2 in.) round","AISI 8622H Steel, heat treated, machined to 25 mm (1 in.) rounds, pseudocarburized, 150°C (300°F) temper","AISI 4140H Steel, normalized at 870°C (1600°F), air cooled, 100 mm (4 in.) round","AISI 1095 Steel, normalized at 900°C (1650°F), air cooled, 100 mm (4 in.) round","AISI 86B30H Steel, water quenched 845°C (1550°F), 540°C (1000°F) temper, 13 mm (0.5 in.) round","AISI Type S21904 (Alloy 21-6-9) Stainless Steel, 102 by 102 mm section annealed bar, tested in longitudinal direction","AISI Type 309S Stainless Steel, annealed, plate","AISI 1144 Steel, normalized at 900°C (1650°F), air cooled, 100 mm (4 in.) round","AISI 1050 Steel, oil quenched 845°C